In [ ]:
# Importing libraries
import sys
import pathlib
import warnings
import numpy as np
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.pyplot as plt
from skimage.io import imread
#from  matplotlib.ticker import FuncFormatter
#from matplotlib_scalebar.scalebar import ScaleBar
from skimage.filters import gaussian
from skimage import img_as_float64, img_as_uint
font_props = {'size': 16}
warnings.filterwarnings("ignore")

In [ ]:
# Defining directories
current_dir = pathlib.Path().absolute()
fa_dir = current_dir.parents[1].joinpath('src')
# Importing fish_analyses module
sys.path.append(str(fa_dir))
import fish_analyses as fa
# Path to credentials
desktop_path = pathlib.Path.home()/'Desktop'
path_to_config_file = desktop_path.joinpath('config.yml')

In [ ]:
data_folder_path =desktop_path.joinpath('FISH_Processing','dataBases','20190909_u2os_multiplex','smFLAG-KDM5B','MAX_Cell01.tif')
# plasmid image located here 'publications/UQ_Bio_2024/images_for_notebooks/pUB_smFLAG-KDM5B-MS2.png'

In [ ]:
# Load image
video = imread(str(data_folder_path))
print(video.shape)
# Color channels used for paper [0,1] 
# Ch 0 = 561 # mRNA
# Ch1 488 # KDM5B


In [ ]:
# show a given time point of the image for a given color channel. Use red and Green channels
def plot_image(video, selected_time_point, selected_channel, selected_colormap, max_percentile, min_percentile,selected_x_range=None,selected_y_range=None, microns_per_pixel=0.13,show_scalebar=True,show_box=False, show_zoom_in=False):
    if not (selected_x_range is None) and (show_zoom_in == True):
        temp_video = video[selected_time_point,selected_x_range[0]:selected_x_range[1] ,selected_y_range[0]:selected_y_range[1],selected_channel]
    else:
        temp_video = video[selected_time_point,: ,:,selected_channel] 
    # Using a gaussian filter to smooth the image
    temp_video = img_as_uint(gaussian(img_as_float64(temp_video), sigma=1))
    max_visualization_value = np.percentile(temp_video,max_percentile)
    min_visualization_value = np.percentile(temp_video, min_percentile)
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.imshow( temp_video ,cmap = selected_colormap, vmin=min_visualization_value, vmax=max_visualization_value)
    # Plotting a yellow box around the selected region
    if show_box ==True:
        ax.plot([selected_y_range[0], selected_y_range[1]], [selected_x_range[0], selected_x_range[0]], '#808080', lw=4)
        ax.plot([selected_y_range[0], selected_y_range[1]], [selected_x_range[1], selected_x_range[1]], '#808080', lw=4)
        ax.plot([selected_y_range[0], selected_y_range[0]], [selected_x_range[0], selected_x_range[1]], '#808080', lw=4)
        ax.plot([selected_y_range[1], selected_y_range[1]], [selected_x_range[0], selected_x_range[1]], '#808080', lw=4)
    if show_scalebar:
        scalebar = ScaleBar(dx = microns_per_pixel, units= 'um', length_fraction=0.25,location='lower right',box_color='k',color='w', font_properties=font_props)
        ax.add_artist(scalebar)
    ax.axis('off')
    plt.show()

In [ ]:
# Selected ranges and time points
selected_time_point = 0 # times used for paper [0]
selected_x_range = [350,400]
selected_y_range = [350,400]


In [ ]:
plot_image(video, 
            selected_time_point=selected_time_point, 
            selected_channel=0, 
            selected_colormap= 'Reds_r', 
            max_percentile=99.9, 
            min_percentile=1,
            selected_x_range = selected_x_range,
            selected_y_range = selected_y_range,
            show_scalebar=True,
            show_box=True,
            show_zoom_in= False,
           )

In [ ]:
plot_image(video, 
            selected_time_point=selected_time_point, 
            selected_channel=1, 
            selected_colormap= 'Greens_r', 
            max_percentile=99.9, 
            min_percentile=1,
            selected_x_range = selected_x_range,
            selected_y_range = selected_y_range,
            show_scalebar=True,
            show_box=True,
            show_zoom_in= False,
           )

In [ ]:
# Third Color channel contains no information.
plot_image(video, 
            selected_time_point=selected_time_point, 
            selected_channel=2, 
            selected_colormap= 'Blues_r', 
            max_percentile=99.9, 
            min_percentile=1,
            selected_x_range = selected_x_range,
            selected_y_range = selected_y_range,
            show_scalebar=True,
            show_box=True,
            show_zoom_in= False,
           )

In [ ]:
# plotting the subsection for channel 0
plot_image(video, 
            selected_time_point=selected_time_point, 
            selected_channel=1, 
            selected_colormap= 'Reds_r', 
            max_percentile=99.9, 
            min_percentile=1,
            selected_x_range=selected_x_range,
            selected_y_range=selected_y_range,
            show_scalebar=False,
            show_zoom_in= True,
           )

In [ ]:
# plotting the subsection for channel 0
plot_image(video, 
           selected_time_point=selected_time_point, 
           selected_channel=1, 
           selected_colormap= 'Greens_r', 
           max_percentile=99.9, 
           min_percentile=5,
           selected_x_range=selected_x_range,
           selected_y_range=selected_y_range,
           show_scalebar=False,
           show_zoom_in= True,
           )

# Image processing image

In [ ]:
# show a given time point of the image for a given color channel. Use red and Green channels
def plot_image_rgb(video, selected_time_point, max_percentile, min_percentile, selected_x_range=None,selected_y_range=None, microns_per_pixel=0.13,show_scalebar=True,show_box=False, show_zoom_in=False, use_gaussian_filter=True):
    if not (selected_x_range is None) and (show_zoom_in == True):
        temp_section = video[selected_time_point,selected_x_range[0]:selected_x_range[1] ,selected_y_range[0]:selected_y_range[1],:]
    else:
        temp_section= video[selected_time_point,: ,:,:] 
    
    if use_gaussian_filter:
        # Using a gaussian filter to smooth the image
        number_channels = video.shape[3]
        for i in range(number_channels):
            temp_section[:,:,i] = img_as_uint(gaussian(img_as_float64(temp_section[:,:,i]), sigma=1))
    temp_video=  fa.Utilities().convert_to_int8(image=temp_section ,min_percentile=min_percentile, max_percentile=max_percentile
    # Plotting
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.imshow( temp_video)
    #ax.imshow( temp_video, vmin=min_visualization_value, vmax=max_visualization_value)
    # Plotting a yellow box around the selected region
    if show_box ==True:
        ax.plot([selected_y_range[0], selected_y_range[1]], [selected_x_range[0], selected_x_range[0]], '#808080', lw=4)
        ax.plot([selected_y_range[0], selected_y_range[1]], [selected_x_range[1], selected_x_range[1]], '#808080', lw=4)
        ax.plot([selected_y_range[0], selected_y_range[0]], [selected_x_range[0], selected_x_range[1]], '#808080', lw=4)
        ax.plot([selected_y_range[1], selected_y_range[1]], [selected_x_range[0], selected_x_range[1]], '#808080', lw=4)
    if show_scalebar:
        scalebar = ScaleBar(dx = microns_per_pixel, units= 'um', length_fraction=0.25,location='lower right',box_color='k',color='w', font_properties=font_props)
        ax.add_artist(scalebar)
    ax.axis('off')
    plt.show()

In [ ]:
plot_image_rgb(video, 
            selected_time_point=selected_time_point, 
            max_percentile=99.9, 
            min_percentile=1,
            selected_x_range = selected_x_range,
            selected_y_range = selected_y_range,
            show_scalebar=True,
            show_box=True,
            show_zoom_in= False,
            use_gaussian_filter=False
           )